In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [ ]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [ ]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "censored"
    consumer_secret = "censored"
    access_token = "censored"
    access_token_secret = "censored"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [#-10.68747448774834, 50.63867628531592, 0.8184988411602326, 58.380364290387824,  # The UK , 
                 110.11672557434858, -37.79116182236319, 152.12789241835733, -11.5066289531987,  # Australia , 
                # -160.161542, 18.776344, -154.641396, 22.878623 # Hawaii
                 ]  

    stream_listener = StreamListener(time_limit=300, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517385593390792705, 'username': 'mohsin_speaks', 'created_at': 'Fri Apr 22 06:11:31 +0000 2022', 'lng': 150.931975, 'lat': -33.848244, 'text': 'Best one from the lot.'}
{'id': 1517385629633433600, 'username': 'MichaelStylian9', 'created_at': 'Fri Apr 22 06:11:40 +0000 2022', 'lng': 150.931975, 'lat': -33.848244, 'text': '@MIGHTYSPURZ Good afternoon from a wet Sydney #COYS'}
{'id': 1517385640895455232, 'username': 'deirdre_mcq', 'created_at': 'Fri Apr 22 06:11:43 +0000 2022', 'lng': 115.9283185, 'lat': -32.1501005, 'text': '@pearlparky Why do they hassle the underemployed more than the unemployed?I was safely unemployed for six years bu… https://t.co/srMpQa6G0q'}
{'id': 1517385691944001536, 'username': 'MillenialSinGrl', 'created_at': 'Fri Apr 22 06:11:55 +0000 2022', 'lng': 150.931975, 'lat': -33.848244, 'text': '@my2ndsinglelife It was opened months ago 🤦🏻\u200d♀️\U0001f972'}
{'id': 1517385712588705792, 'username': 'MacMate_', 'created_at': 'Fri Apr 22 06:12:00 +0000 2022', 'l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
